<a href="https://colab.research.google.com/github/AugustvonMackensen/AI_colab/blob/main/3_CNN_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3_CNN_torch.ipynb**

# **CNN(Convolutional Neural Network)**

- MNIST data
- 3개 합성곱 레이어
- 2개 풀링 레이어

# **1. Settings**
**1) import required libraries**

In [2]:
# !pip3 install visdom

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

# from visdom import Visdom
# viz = Visdom()

**2) Set hyperparameters**

In [4]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

# **2. Data**
**1) Download Data**

In [5]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

**2) Check Dataset**

In [6]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


**3) Set DataLoader**

In [7]:
# train_loader = torch.utils.data.DataLoader()
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# **3. Model & Optimizer**
**1) CNN Model**

In [8]:
from torch.nn.modules.pooling import MaxPool2d
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 5),
        nn.ReLU(),
        nn.Conv2d(16, 32, 5),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, 5),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)

    return out

In [9]:
model = CNN().cuda()

**2) Loss func & Optimizer**

In [10]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# **4. Train**

In [11]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(train_loader):
    x = Variable(image).cuda()
    y_ = Variable(label).cuda()

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()
    
    if j % 1000 == 0:
      print(loss)
  

tensor(2.3102, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3095, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3093, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2818, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2984, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3032, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2770, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2985, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3098, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2792, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2728, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2540, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2478, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2601, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2382, device='cuda:0', grad_fn=

In [12]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[-4.4416e-02,  1.4216e-01,  7.6905e-02,  1.4650e-01, -5.5217e-05],
          [-5.0251e-02, -1.0379e-01, -1.4513e-01, -2.0031e-01,  1.9314e-02],
          [-7.7925e-02, -1.1359e-01, -1.1453e-01, -1.5375e-01,  9.8473e-02],
          [ 1.3474e-01,  5.5373e-02,  1.4828e-01, -9.0616e-02, -2.5754e-02],
          [-1.5564e-01,  1.5755e-01, -3.1844e-02, -8.7853e-02,  1.8834e-01]]],


        [[[ 1.3100e-01,  1.6957e-01, -2.5379e-02,  1.9523e-01,  2.0510e-02],
          [-1.2787e-01,  1.4633e-01, -1.0217e-01,  1.2219e-01, -1.9654e-02],
          [-9.3791e-02, -3.7351e-02,  4.0991e-02, -1.9429e-01, -2.6179e-02],
          [-1.5288e-01,  1.4556e-01, -1.3333e-01,  2.0257e-02, -8.6113e-02],
          [ 9.2630e-02, -1.5888e-01, -9.0756e-02,  6.5368e-02, -1.7713e-01]]],


        [[[-1.1251e-01,  1.3627e-02,  9.2248e-02, -1.0074e-01, -6.8862e-02],
          [-1.7622e-01, -1.4483e-01,  1.4698e-01,  1.4809e-02,  2.8448e-02],
          [-6.2578e-03,  1.1237e-01, -1.4072e

# **5. Test**

In [14]:
correct = 0
total = 0

for image, label in test_loader:
  x = Variable(image, volatile=True).cuda()
  y_ = Variable(label).cuda()

  output = model.forward(x)
  _, output_index = torch.max(output, 1)

  total += label.size(0)
  correct += (output_index == y_).sum().float()

print('평가 데이터 정확도 : {}'.format(100 * correct / total))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


평가 데이터 정확도 : 89.91999816894531
